<a href="https://colab.research.google.com/github/fontesdearaujo/text_treatment/blob/master/PigeonXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from pigeonXT import annotate
annotations = annotate(
  ['I love this movie', 'I was really disappointed by the book'],
  options=['positive', 'negative'])

HTML(value='0 examples annotated, 3 examples left')

Output()

Annotation done.


In [11]:
from pigeonXT import annotate
import pandas as pd
import numpy as np
from pathlib import Path


df = pd.DataFrame([
    {'title': 'Star wars'},
    {'title': 'The Positively True Adventures of the Alleged Texas Cheerleader-Murdering Mom'},
    {'title': 'Eternal Sunshine of the Spotless Mind'},
    {'title': 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb'},
    {'title': 'Killer klowns from outer space'},
])

labels = ['Adventure', 'Romance', 'Fantasy', 'Science fiction', 'Horror', 'Thriller']
shortLabels = ['A', 'R', 'F', 'SF', 'H', 'T']

df.to_csv('inputtestdata.csv', index=False)


def setLabels(labels, numClasses):
    row = np.zeros([numClasses], dtype=np.uint8)
    row[labels] = 1
    return row

def labelPortion(inputFile,
                 labels = ['yes', 'no'],
                 outputFile='output.csv',
                 portionSize=2,
                 textColumn='title',
                 shortLabels=None):
    if shortLabels == None:
        shortLabels = labels
    out = Path(outputFile)
    if out.exists():
        outdf = pd.read_csv(out)
        currentId = outdf.index.max() + 1
    else:
        currentId = 0
    indf = pd.read_csv(inputFile)
    examplesInFile = len(indf)
    indf = indf.loc[currentId:currentId + portionSize - 1]
    actualPortionSize = len(indf)
    print(f'{currentId + 1} - {currentId + actualPortionSize} of {examplesInFile}')
    sentences = indf[textColumn].tolist()

    for label in shortLabels:
        indf[label] = None

    def updateRow(ix, selectedLabels):
        print(ix, selectedLabels)
        labs = setLabels([labels.index(y) for y in selectedLabels], len(labels))
        indf.loc[indf.index.min()+ix, shortLabels] = labs

    def finalProcessing(annotations):
        if out.exists():
            prevdata = pd.read_csv(out)
            outdata = pd.concat([prevdata, indf]).reset_index(drop=True)
        else:
            outdata = indf.copy()
        outdata.to_csv(out, index=False)

    annotated = annotate( sentences,
                          options=labels,
                          task_type='multilabel-classification',
                          buttons_in_a_row=3,
                          reset_buttons_after_click=True,
                          include_skip=False,
                          example_process_fn=updateRow,
                          final_process_fn=finalProcessing)
    return indf


annotations = labelPortion('inputtestdata.csv',
                           labels=labels,
                           shortLabels= shortLabels)

1 - 2 of 5


HTML(value='0 examples annotated, 3 examples left')

Output()

0 ['Adventure', 'Science fiction']
1 ['Horror']
Annotation done.
